In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/.gitattributes
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00002-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/gener

In [2]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)


In [3]:

model_id = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
# Load the test dataset
test_data = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet')

messages_pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

# Prepare messages for prediction
def prepare_message(row):
    return (
        "Choose the better response. Answer with 'model_a' or 'model_b'. "
        "Only provide one word as your answer.\n\nInstruction:\n"
        "Given the following responses to the prompt, choose which one is better.\n\n"
        f"Input:\n{row['prompt']}\n\n"
        f"Response A:\n{row['response_a']}\n\n"
        f"Response B:\n{row['response_b']}\n\n"
        "Which response is better?\nmodel_a or model_b\n\nAnswer with only one word."
    )


# Predict for each row in the test dataset
predictions = []
for _, row in test_data.iterrows():
    messages = prepare_message(row)
    output = messages_pipeline(messages, max_new_tokens=256)
    
    # Extract generated text and handle it properly
    if isinstance(output, list) and "generated_text" in output[0]:
        generated_text = output[0]["generated_text"].strip().lower()  # Normalize
        prediction = "model_a" if "model_a" in generated_text else "model_b"
    else:
        prediction = "model_b"  # Default fallback

    
    # Ensure valid prediction
    if prediction not in ["model_a", "model_b"]:
        prediction = "model_a"  # Fallback for invalid predictions
    
    predictions.append({"id": row['id'], "winner": prediction})

# Convert predictions to a DataFrame
submission = pd.DataFrame(predictions)

# Save to a CSV file in the required submission format
submission.to_csv("submission.csv", index=False)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [4]:
import pandas as pd

view = pd.read_csv('/kaggle/working/submission.csv')
view.head()

,id,winner
0,327228,model_a
1,1139415,model_a
2,1235630,model_a


In [5]:
# # Load and preprocess dataset
# train_data = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet')

# # Prepare data for training
# def prepare_data(row):
#     return {
#         "prompt": row["prompt"],
#         "response_a": row["response_a"],
#         "response_b": row["response_b"],
#         "winner": row["winner"]  # Use this if it's needed for labels
#     }

# train_data_prepared = train_data.apply(prepare_data, axis=1).tolist()
# train_dataset = Dataset.from_pandas(pd.DataFrame(train_data_prepared))

# # Define tokenizer and model
# model_name = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Enable gradient checkpointing to save memory
# model.gradient_checkpointing_enable()

# # Tokenize dataset with smaller max length
# def preprocess_data(example):
#     return tokenizer(example['prompt'], text_pair=example['response_a'], truncation=True, max_length=256)  # Lower max_length

# tokenized_dataset = train_dataset.map(preprocess_data, batched=True)

# # Fine-tuning configurations with multi-GPU setup
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="steps",
#     save_steps=500,
#     per_device_train_batch_size=1,  # Lower batch size
#     per_device_eval_batch_size=1,
#     num_train_epochs=1,
#     learning_rate=1e-3,
#     save_total_limit=2,
#     logging_dir="./logs",
#     logging_steps=100,
#     fp16=True,  # Use mixed precision for faster training
#     device_map="auto",  # Automatically use available GPUs
# )

# # Setup the Trainer with the multi-GPU configuration
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset,
# )

# # Train model
# trainer.train()


In [6]:
# # After training, perform inference directly with the trained model
# messages_pipeline = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto",  # This ensures it's running on the available GPU (if applicable)
# )

# # Load the test dataset
# test_data = pd.read_parquet('/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet')

# # Prepare messages for prediction
# def prepare_message(row):
#     return [
#         {"role": "system", "content": "Choose the better response. Answer with 'model_a' or 'model_b'. Only provide one word as your answer.\n\nInstruction:\nGiven the following responses to the prompt, choose which one is better.\n\nInput:\n" 
#          + row['prompt'] +
#          "\n\nResponse A:\n" + row['response_a'] +
#          "\n\nResponse B:\n" + row['response_b'] +
#          "\n\nWhich response is better?\nmodel_a or model_b\n\nAnswer with only one word."}
#     ]

# # Predict for each row in the test dataset
# predictions = []
# for _, row in test_data.iterrows():
#     messages = prepare_message(row)
#     output = messages_pipeline(messages, max_new_tokens=256)
    
#     # Extract generated text and handle it properly
#     if isinstance(output, list) and "generated_text" in output[0]:
#         prediction = output[0]["generated_text"][-1]  # Extract the last character to determine model_a or model_b
#     else:
#         prediction = "model_a"  # Fallback to model_a if the output is unexpected
    
#     # Ensure valid prediction
#     if prediction not in ["model_a", "model_b"]:
#         prediction = "model_a"  # Fallback for invalid predictions
    
#     predictions.append({"id": row['id'], "winner": prediction})
#     print(f"Prediction completed for id {row['id']}")

# # Convert predictions to a DataFrame
# submission = pd.DataFrame(predictions)

# # Save to a CSV file in the required submission format
# submission.to_csv("submission.csv", index=False)